Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [3]:
#Ваш код здесь
f_m = df.loc[df['gender']=='Male', 'gender'].count()/df.loc[df['gender']=='Female', 'gender'].count()
print(f'Мужчин в {f_m} раз больше, чем женщин') if f_m > 1 else print(f'Женщин больше в {1/f_m} раз, чем мужчин')

Мужчин в 1.0192087155963303 раз больше, чем женщин


##### 2. Какое количество уникальных значений у поля InternetService?

In [4]:
#Ваш код здесь
internet_unique = df['InternetService'].value_counts().index
print(f'У поля InternetService {len(internet_unique)} уникальных значений.\nУникальные значения: {list(internet_unique)}')

У поля InternetService 3 уникальных значений.
Уникальные значения: ['Fiber optic', 'DSL', 'No']


##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [9]:
#Ваш код здесь
df['TotalCharges'].describe()

count     7043
unique    6531
top       20.2
freq        11
Name: TotalCharges, dtype: object

В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

In [11]:
# У данного поля тип object, поэтому вывести статистики нельзя

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [12]:
#Ваш код здесь
binary_to_numbers = {'Yes': 1, 'No': 0}
df['PhoneService'] = df['PhoneService'].map(binary_to_numbers)
df['PhoneService'].value_counts()

1    6361
0     682
Name: PhoneService, dtype: int64

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [15]:
#Ваш код здесь
df.loc[df['TotalCharges']==' ', 'TotalCharges'] = np.nan
df['TotalCharges'] = df['TotalCharges'].astype(np.float32)
df['TotalCharges'].fillna(0, inplace=True)
df[['TotalCharges']].describe()

,TotalCharges
count,7043.000000
mean,2279.732178
std,2266.794434
min,0.000000
25%,398.549988
50%,1394.550049
75%,3786.599976
max,8684.799805


##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [17]:
#Ваш код здесь
df['Churn'] = df['Churn'].map(binary_to_numbers)
df['Churn'].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [21]:
#Ваш код здесь
binary_to_numbers2 = {'Yes': 1, 'No': 0, 'No internet service': 0}
df['StreamingMovies'] = df['StreamingMovies']. map(binary_to_numbers2)
df['StreamingTV'] = df['StreamingTV']. map(binary_to_numbers2)
df['TechSupport'] = df['TechSupport']. map(binary_to_numbers2)

##### 8. Заполните пропуски в поле PhoneService значением 0

In [25]:
#Ваш код здесь
df['PhoneService'].fillna(0, inplace=True)

#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [27]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']
#Ваш код здесь
df = df[columns]
df.head(3)


,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,Female,1,0,29.850000,0,0,0,0
1,Male,34,1,1889.500000,0,0,0,0
2,Male,2,1,108.150002,0,0,0,1


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [28]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'
#Ваш код здесь
X = df.drop(columns=target)
y = df[target]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, 
                                                      shuffle=True, 
                                                      stratify=y, 
                                                      random_state=42)

##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [29]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [30]:
#Ваш код здесь
gender.fit_transform(X_train)


,gender_Female,gender_Male
5557,1,0
2270,1,0
6930,1,0
2257,1,0
898,1,0
...,...,...
4250,0,1
1488,0,1
6303,1,0
2710,1,0


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [31]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [32]:
#Ваш код здесь
TotalCharges =  Pipeline([
    ('selector', NumberSelector(key='TotalCharges')),
    ('standard', StandardScaler())
])

Объединение всех "кубиков" очень легко сделать таким образом

In [33]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [34]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [35]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('tenure',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler(copy=True,
                                                                                 with_mean=True,
                                                                                 with_std=True))],
                                                          verbose=False)),
                                                ('TotalCharges',
                                                 Pipeline(memory=None,
                        

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [37]:
#Ваш код здесь
pred = pipeline.predict_proba(X_valid)
pred

array([[0.83, 0.17],
       [0.99, 0.01],
       [0.21, 0.79],
       ...,
       [0.96, 0.04],
       [0.87, 0.13],
       [1.  , 0.  ]])

##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [40]:
from sklearn.metrics import roc_auc_score, log_loss

#Ваш код здесь
print(f'logloss = {log_loss(y_valid, pred[:, 1])}')
print(f'roc_auc = {roc_auc_score(y_valid, pred[:, 1])}')

logloss = 0.8258392061051559
roc_auc = 0.7630181055552492


### Сохраним наш пайплайн

In [41]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)